### ¿Qué es un adaptador en WDF?

En los WDF, los **adaptadores** permiten conectar varios elementos (como resistores y capacitores) de forma que se cumplan las leyes de Kirchhoff y se mantenga la pasividad del sistema.

Los dos adaptadores básicos son:
- **Adaptador serie**: garantiza que la corriente sea la misma entre dos elementos conectados en serie.
- **Adaptador paralelo**: garantiza que el voltaje sea el mismo entre dos elementos en paralelo.

Estos adaptadores se basan en el concepto de **resistencia de puerto (Rp)** y la transformación de variables de voltaje/corriente a ondas incidentes/reflejadas.


### Fórmulas del adaptador serie

Dado dos elementos con resistencias de puerto \( R_1 \) y \( R_2 \):

- **Resistencia total**:

$$
R_p = R_1 + R_2
$$

- **Coeficiente de reflexión relativo**:

$$
\beta = \frac{R_1}{R_1 + R_2}
$$

- **Ondas reflejadas**:

$$
b_1 = a_2 + \beta(a_1 - a_2), \quad b_2 = a_1 + \beta(a_2 - a_1)
$$

La implementación en `pywdf` de un adaptador serie se realiza con la clase `SeriesAdaptor`.



In [2]:
from pywdf.core.wdf import Resistor, SeriesAdaptor

# Crear dos resistores
R1 = Resistor(1000)  # 1k Ohm
R2 = Resistor(1000)  # 1k Ohm

# Conectarlos con adaptador serie
S = SeriesAdaptor(R1, R2)

# Simular ondas incidentes
S.accept_incident_wave(1.0)
S.propagate_reflected_wave()

# Mostrar resultados
print("a1:", R1.a, "| b1:", R1.b)
print("a2:", R2.a, "| b2:", R2.b)
print("Serie Rp:", S.Rp)


a1: -0.5 | b1: 0
a2: -0.5 | b2: 0
Serie Rp: 2000


### Fórmulas del adaptador paralelo

Dado dos elementos con conductancias \( G_1 = 1/R_1 \), \( G_2 = 1/R_2 \):

- **Conductancia total**:

$$
G = G_1 + G_2, \quad R_p = \frac{1}{G}
$$

- **Coeficiente**:

$$
\beta = \frac{G_1}{G_1 + G_2}
$$

- **Ondas reflejadas**:

Se calculan a partir de la diferencia de \( b \)'s (según el principio dual al adaptador serie).

La clase en `pywdf` que implementa esto es `ParallelAdaptor`.



In [3]:
from pywdf.core.wdf import ParallelAdaptor

# Crear dos resistores
R1 = Resistor(1000)
R2 = Resistor(1000)

# Conectarlos con adaptador paralelo
P = ParallelAdaptor(R1, R2)

# Simular ondas
P.accept_incident_wave(1.0)
P.propagate_reflected_wave()

# Mostrar resultados
print("a1:", R1.a, "| b1:", R1.b)
print("a2:", R2.a, "| b2:", R2.b)
print("Paralelo Rp:", P.Rp)


a1: 1.0 | b1: 0
a2: 1.0 | b2: 0
Paralelo Rp: 500.0


Podemos verificar que para valores positivos de \( R_1, R_2 \), el sistema es pasivo:
- No genera energía.
- La potencia reflejada siempre es menor o igual a la incidente:
$$
P = a^2 - b^2 \geq 0
$$


In [4]:
import numpy as np

a_wave = 1.0
P.accept_incident_wave(a_wave)
b_wave = P.propagate_reflected_wave()

P_total = a_wave**2 - b_wave**2
print("Potencia neta (a^2 - b^2):", P_total)


Potencia neta (a^2 - b^2): 1.0


### ¿Qué pasa si conectamos más de 2 elementos?

Para adaptadores de 3 o más elementos, se usan estructuras llamadas `RTypeAdaptor`, que calculan las relaciones de ondas usando una matriz de dispersión \( \mathbf{S} \).

Esto lo veremos en detalle en el notebook 06, cuando construyamos árboles de conexión complejos.

Por ahora, sabemos que:

- `RTypeAdaptor` permite conexiones arbitrarias
- `pywdf` tiene clases como `RootRTypeAdaptor` para manejar el nodo raíz

